# Step 1 clone github

In [7]:
!git clone https://github.com/SarayutMI/YOLO_Objecdetection.git

Cloning into 'YOLO_Objecdetection'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 54 (delta 0), reused 0 (delta 0), pack-reused 52 (from 1)
Receiving objects: 100% (54/54), 107.61 MiB | 36.99 MiB/s, done.
Resolving deltas: 100% (5/5), done.


# Step2 install Env ultralytics

In [8]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

# Step 3 Import Libary and **Code**

In [9]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob
import random
import os

# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))

        cv2.rectangle(
            image,
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image


# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))

    all_images.sort()

    num_images = len(all_images)

    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()



# Step 4 Prediction Model [ Folder Image All Class ] + Check Distace

In [11]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
from ultralytics import YOLO
import os

# โหลดโมเดลที่เทรนเสร็จแล้ว
model = YOLO("/content/YOLO_Objecdetection/yolov8n_v8_50e4/weights/best.pt")

# โฟลเดอร์ที่เก็บภาพ
input_folder = "/content/YOLO_Objecdetection/inputimage"
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# ฟังก์ชันคำนวณระยะห่าง
def calculate_distance(box1, box2):
    x1, y1, x2, y2 = box1
    a1, b1, a2, b2 = box2
    center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
    center2 = ((a1 + a2) / 2, (b1 + b2) / 2)
    return np.linalg.norm(np.array(center1) - np.array(center2))

# ระยะที่ถือว่า "ใกล้"
THRESHOLD_DISTANCE = 170

# วนลูปประมวลผลภาพทั้งหมด
for img_file in image_files:
    img_path = os.path.join(input_folder, img_file)
    results = model.predict(img_path, conf=0.25)

    for result in results:
        img = cv2.imread(img_path)
        boxes = result.boxes

        # แยกข้อมูล class ต่างๆ
        persons = []
        objects = {"brush": [], "bucket": [], "cigarette": [], "smoke": [], "roller": []}

        for box in boxes:
            xyxy = box.xyxy[0].cpu().numpy().astype(int)
            class_id = int(box.cls[0].cpu().numpy())
            conf = box.conf[0].cpu().numpy()
            class_name = model.names[class_id]

            if class_name == "person":
                persons.append(xyxy)
            elif class_name in objects:
                objects[class_name].append(xyxy)

            # วาดกรอบทุก class
            label = f"{class_name} {conf:.2f}"
            cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
            cv2.putText(img, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # ตรวจสอบระยะห่างระหว่าง "person" กับวัตถุ
        for px1, py1, px2, py2 in persons:
            for obj_class, object_list in objects.items():
                for bx1, by1, bx2, by2 in object_list:
                    distance = calculate_distance((px1, py1, px2, py2), (bx1, by1, bx2, by2))

                    # พิมพ์ระยะห่าง
                    print(f"Distance between person and {obj_class}: {distance:.2f} pixels")

                    if distance < THRESHOLD_DISTANCE:
                        cv2.rectangle(img, (px1, py1), (px2, py2), (255, 0, 0), 2)  # วาดกรอบสีน้ำเงิน
                        cv2.putText(img, f"Near {obj_class}", (px1, py1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # แสดงภาพใน Google Colab
        cv2_imshow(img)


Output hidden; open in https://colab.research.google.com to view.

# Step 4 Prediction Model [ Folder Image ]

In [ ]:
import cv2
import torch
import os
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# โหลดโมเดลที่เทรนไว้
model = YOLO("/content/YOLO_Objecdetection/yolov8n_v8_50e4/weights/best.pt")
conf_threshold = 0.8  # ค่าความมั่นใจที่ใช้สำหรับกรองผลทำนาย

# ตั้งค่าโฟลเดอร์ที่มีภาพ
input_folder = "/content/YOLO_Objecdetection/inputimage"

# อ่านไฟล์ทั้งหมดในโฟลเดอร์
for filename in os.listdir(input_folder):
    image_path = os.path.join(input_folder, filename)

    # ตรวจสอบว่าไฟล์เป็นภาพ
    if image_path.endswith(('.jpg', '.jpeg', '.png')):
        # อ่านภาพ
        image = cv2.imread(image_path)

        # ตรวจสอบว่าภาพถูกโหลดหรือไม่
        if image is None:
            print(f"Error loading image {filename}.")
            continue

        # รันการตรวจจับ
        results = model(image)

        # อ่านผลลัพธ์
        for result in results:
            boxes = result.boxes.xyxy  # (x1, y1, x2, y2)
            labels = result.boxes.cls  # Class Index

            persons = []
            objects = {"brush": [], "bucket": [], "roller": [], "cigarette": [], "smoke": []}

            # แยกข้อมูล class ต่างๆ
            for i, label in enumerate(labels):
                x1, y1, x2, y2 = map(int, boxes[i])
                class_name = model.names[int(label)]

                if class_name == "person":
                    persons.append((x1, y1, x2, y2))  # เก็บพิกัดบุคคล
                elif class_name in objects:
                    objects[class_name].append((x1, y1, x2, y2))

            # ตรวจจับการกระทำของ "person"
            for px1, py1, px2, py2 in persons:
                is_painting = False
                is_smoking = False

                # เช็คว่าพบ "brush" หรือ "bucket" หรือ "roller" ใกล้กับ "person"
                for bx1, by1, bx2, by2 in objects["brush"] + objects["bucket"] + objects["roller"]:
                    if bx1 < px2 and bx2 > px1 and by1 < py2 and by2 > py1:
                        is_painting = True
                        break  # เจออันใดอันหนึ่งก็พอ

                # เช็คว่าพบ "cigarette" หรือ "smoke" ใกล้กับ "person"
                for cx1, cy1, cx2, cy2 in objects["cigarette"] + objects["smoke"]:
                    if cx1 < px2 and cx2 > px1 and cy1 < py2 and cy2 > py1:
                        is_smoking = True
                        break  # เจออันใดอันหนึ่งก็พอ

                # กำหนดการกระทำของบุคคล
                if is_painting:
                    action_label = "Person Painting"
                    color = (0, 255, 0)  # สีเขียว
                elif is_smoking:
                    action_label = "Person Smoking"
                    color = (0, 0, 255)  # สีแดง
                else:
                    action_label = "Person NoAction"
                    color = (255, 0, 0)  # สีน้ำเงิน

                # วาดกรอบที่บุคคล (person)
                cv2.rectangle(image, (px1, py1), (px2, py2), color, 2)
                # วาดข้อความ
                cv2.putText(image, action_label, (px1, py1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # แสดงผลภาพ
        cv2_imshow(image)


Output hidden; open in https://colab.research.google.com to view.

# Step 5 Prediction Model [ Folder Video ]

In [ ]:
import cv2
import torch
from ultralytics import YOLO
import os
import numpy as np
from IPython.display import HTML
from base64 import b64encode

# โหลดโมเดล
model = YOLO("/content/YOLO_Objecdetection/yolov8n_v8_50e4/weights/best.pt")

# ตั้งค่าพารามิเตอร์
CONFIDENCE_THRESHOLD = 0.34
THRESHOLD_DISTANCE = 850

# กำหนดโฟลเดอร์ต้นทางและปลายทาง
input_folder = "/content/YOLO_Objecdetection/inputVideo"
output_folder = "/content/YOLO_Objecdetection/output_inputVideo"

# สร้างโฟลเดอร์ปลายทางหากยังไม่มี
os.makedirs(output_folder, exist_ok=True)

# ฟังก์ชันคำนวณระยะห่างระหว่างจุดศูนย์กลางของสองกล่อง
def calculate_distance(box1, box2):
    x1, y1, x2, y2 = box1
    a1, b1, a2, b2 = box2
    center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
    center2 = ((a1 + a2) / 2, (b1 + b2) / 2)
    return np.linalg.norm(np.array(center1) - np.array(center2))

# วนลูปผ่านวิดีโอทุกไฟล์ในโฟลเดอร์
for filename in os.listdir(input_folder):
    if filename.lower().endswith((".mp4", ".mov", ".avi")):
        video_path = os.path.join(input_folder, filename)
        save_path = os.path.join(output_folder, filename.replace(".MOV", ".avi").replace(".mp4", ".avi"))

        # เปิดไฟล์วิดีโอ
        cap = cv2.VideoCapture(video_path)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # ตั้งค่า VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(save_path, fourcc, 20.0, (frame_width, frame_height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame)

            for result in results:
                boxes = result.boxes.xyxy
                labels = result.boxes.cls
                confidences = result.boxes.conf

                persons = []
                objects = {"brush": [], "bucket": [], "roller": [], "cigarette": [], "smoke": []}

                for i, label in enumerate(labels):
                    x1, y1, x2, y2 = map(int, boxes[i])
                    conf = float(confidences[i])
                    class_name = model.names[int(label)]

                    if class_name == "person" and conf >= CONFIDENCE_THRESHOLD:
                        persons.append((x1, y1, x2, y2))
                    elif class_name in objects:
                        objects[class_name].append((x1, y1, x2, y2))

            for px1, py1, px2, py2 in persons:
                is_painting = False
                is_smoking = False

                for obj_class, object_list in objects.items():
                    for bx1, by1, bx2, by2 in object_list:
                        distance = calculate_distance((px1, py1, px2, py2), (bx1, by1, bx2, by2))

                        max_distance = 1300 if obj_class == "roller" else THRESHOLD_DISTANCE

                        if distance < max_distance:
                            if obj_class in ["brush", "bucket", "roller"]:
                                is_painting = True
                            elif obj_class in ["cigarette", "smoke"]:
                                is_smoking = True

                if is_painting:
                    action_label = "Person Painting"
                    color = (0, 255, 0)
                elif is_smoking:
                    action_label = "Person Smoking"
                    color = (0, 0, 255)
                else:
                    action_label = "Person NoAction"
                    color = (255, 0, 0)

                cv2.rectangle(frame, (px1, py1), (px2, py2), color, 2)
                cv2.putText(frame, action_label, (px1, py1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            out.write(frame)

        cap.release()
        out.release()

        # บีบอัดเป็น MP4
        compressed_path = os.path.join(output_folder, filename.replace(".MOV", ".mp4").replace(".avi", ".mp4"))
        os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

        print(f"✅ ประมวลผลเสร็จ: {filename}")

print("🎉 เสร็จสิ้นการประมวลผลทุกไฟล์ในโฟลเดอร์!")



0: 1280x704 1 cigarette, 1 person, 1443.9ms
Speed: 17.8ms preprocess, 1443.9ms inference, 1.5ms postprocess per image at shape (1, 3, 1280, 704)

0: 1280x704 1 cigarette, 1 person, 2099.2ms
Speed: 22.7ms preprocess, 2099.2ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 704)

0: 1280x704 2 cigarettes, 1 person, 1644.3ms
Speed: 28.5ms preprocess, 1644.3ms inference, 1.5ms postprocess per image at shape (1, 3, 1280, 704)

0: 1280x704 2 cigarettes, 1 person, 1338.4ms
Speed: 15.2ms preprocess, 1338.4ms inference, 1.4ms postprocess per image at shape (1, 3, 1280, 704)

0: 1280x704 1 cigarette, 1 person, 1299.2ms
Speed: 15.3ms preprocess, 1299.2ms inference, 1.1ms postprocess per image at shape (1, 3, 1280, 704)

0: 1280x704 1 cigarette, 1 person, 1463.6ms
Speed: 14.8ms preprocess, 1463.6ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 704)

0: 1280x704 1 cigarette, 1 person, 1399.1ms
Speed: 15.4ms preprocess, 1399.1ms inference, 1.4ms postprocess per image at 

# Step 5 Prediction Model [ Folder Video ]  Just 1 video

In [ ]:
import cv2
import torch
from ultralytics import YOLO
import os
import numpy as np
from IPython.display import HTML
from base64 import b64encode

# โหลดโมเดลที่เทรนไว้
model = YOLO("/content/YOLO_Objecdetection/yolov8n_v8_50e4/weights/best.pt")

CONFIDENCE_THRESHOLD = 0.34  # เพิ่มเงื่อนไขสำหรับ person
THRESHOLD_DISTANCE = 850  # ปรับค่าระยะที่ถือว่าใกล้

# กำหนดโฟลเดอร์ที่เก็บวิดีโอและไฟล์ผลลัพธ์
video_folder = "/content/YOLO_Objecdetection/inputOneVideo"
save_folder = "/content/YOLO_Objecdetection/output_inputOneVideo"

# ตรวจสอบว่าโฟลเดอร์เซฟมีอยู่หรือไม่ ถ้าไม่มีให้สร้างขึ้นมา
os.makedirs(save_folder, exist_ok=True)

# ฟังก์ชันคำนวณระยะห่างระหว่างจุดศูนย์กลางของสองกล่อง
def calculate_distance(box1, box2):
    x1, y1, x2, y2 = box1
    a1, b1, a2, b2 = box2
    center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
    center2 = ((a1 + a2) / 2, (b1 + b2) / 2)
    return np.linalg.norm(np.array(center1) - np.array(center2))

# วนลูปอ่านไฟล์ในโฟลเดอร์
for filename in os.listdir(video_folder):
    if filename.endswith(".MOV") or filename.endswith(".mp4"):  # ตรวจสอบว่าเป็นไฟล์วิดีโอ
        video_path = os.path.join(video_folder, filename)
        save_path = os.path.join(save_folder, filename.replace(".MOV", ".avi").replace(".mp4", ".avi"))

        # เปิดไฟล์วิดีโอ
        cap = cv2.VideoCapture(video_path)

        # รับค่าขนาดเฟรม (width, height)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # กำหนด codec และสร้าง VideoWriter สำหรับบันทึกวิดีโอที่ตรวจจับ
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(save_path, fourcc, 20.0, (frame_width, frame_height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # ถ้าไม่สามารถอ่านเฟรมได้ให้หยุด

            # รันการตรวจจับวัตถุในแต่ละเฟรม
            results = model(frame)

            # อ่านผลลัพธ์
            for result in results:
                boxes = result.boxes.xyxy  # (x1, y1, x2, y2)
                labels = result.boxes.cls  # Class Index
                confidences = result.boxes.conf  # ค่าความมั่นใจ

                persons = []
                objects = {"brush": [], "bucket": [], "roller": [], "cigarette": [], "smoke": []}

                # แยกข้อมูล class ต่างๆ
                for i, label in enumerate(labels):
                    x1, y1, x2, y2 = map(int, boxes[i])
                    conf = float(confidences[i])  # แปลงค่า confidence เป็น float
                    class_name = model.names[int(label)]

                    if class_name == "person" and conf >= CONFIDENCE_THRESHOLD:
                        persons.append((x1, y1, x2, y2))  # เก็บพิกัดบุคคล
                    elif class_name in objects:
                        objects[class_name].append((x1, y1, x2, y2))

            # ตรวจจับการกระทำของ "person"
            for px1, py1, px2, py2 in persons:
                is_painting = False
                is_smoking = False

                # ตรวจสอบวัตถุที่อยู่ใกล้คน
                for obj_class, object_list in objects.items():
                    for bx1, by1, bx2, by2 in object_list:
                        distance = calculate_distance((px1, py1, px2, py2), (bx1, by1, bx2, by2))

                        # กำหนดระยะที่อนุญาตให้วัตถุอยู่ใกล้
                        if obj_class == "roller":
                            max_distance = 1300  # อนุญาตให้ roller อยู่ห่างได้ถึง 1300
                        else:
                            max_distance = THRESHOLD_DISTANCE  # ค่าเริ่มต้น 850

                        if distance < max_distance:  # ถ้าอยู่ในระยะใกล้
                            if obj_class in ["brush", "bucket", "roller"]:
                                is_painting = True
                            elif obj_class in ["cigarette", "smoke"]:
                                is_smoking = True

                # กำหนดการกระทำของบุคคล
                if is_painting:
                    action_label = "Person Painting"
                    color = (0, 255, 0)  # สีเขียว
                elif is_smoking:
                    action_label = "Person Smoking"
                    color = (0, 0, 255)  # สีแดง
                else:
                    action_label = "Person NoAction"
                    color = (255, 0, 0)  # สีน้ำเงิน

                # วาดกรอบที่บุคคล (person)
                cv2.rectangle(frame, (px1, py1), (px2, py2), color, 2)
                # วาดข้อความ
                cv2.putText(frame, action_label, (px1, py1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # เขียนเฟรมที่ตรวจจับแล้วลงในวิดีโอ
            out.write(frame)

        # ปิดไฟล์วิดีโอและ VideoWriter
        cap.release()
        out.release()

        # แปลงเป็น mp4
        compressed_path = save_path.replace(".avi", ".mp4")
        os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

        # อ่านไฟล์วิดีโอที่บีบอัดแล้ว
        mp4 = open(compressed_path, 'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

        # แสดงวิดีโอใน Jupyter Notebook / Colab
        display(HTML(f"""
        <video width=400 controls>
              <source src="{data_url}" type="video/mp4">
        </video>
        """))

print("✅ การประมวลผลวิดีโอเสร็จสมบูรณ์!")
